In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier


In [2]:
reviews_train = pd.read_csv('preprocessed-datasets/Review_prepro.csv')
reviews_train.set_index('ID', inplace=True)

## Function for different feature selection methods

Others should?/can be added. Maybe we should also try them with cross validation to have better results?

In [3]:
def select_features(X, y, method, corr_threshold=0.5):
    
    if method is None:
        return X.columns
        
    # Feature selection using chi-squared
    elif method == 'p-value':
        # Remove constant features
        X = X.loc[:, X.apply(pd.Series.nunique) != 1]
        p_values = []
        for feature in X.columns:
            f, p = chi2(X[[feature]], y)
            p_values.append(p)

        p_values = np.array(p_values).reshape(-1)
        p_values = pd.Series(p_values, index=X.columns).sort_values()
        selected_features = p_values[p_values < 0.05].index
    
    # Feature selection using VarianceThreshold
    elif method == 'variance':
        X = X.loc[:, X.apply(pd.Series.nunique) != 1]
        var_thr = VarianceThreshold(threshold=0.25)
        var_thr.fit(X)
        selected_features = X.columns[var_thr.get_support()]  
    
    # Feature selection using Pearson correlation
    elif method == 'correlation':
        X = X.loc[:, X.apply(pd.Series.nunique) != 1]
        corr = X.corr()
        col_corr = set()

        corr_matrix = np.abs(corr.values)

        n = corr_matrix.shape[0]

        for i in range(n):
            for j in range(i):
                if corr_matrix[i, j] > corr_threshold:
                    colname = X.columns[i]
                    col_corr.add(colname)

        selected_features = [col for col in X.columns if col not in col_corr]

    # More methods to be added 
                
    return selected_features

## Split data into train and test

In [4]:
# Ecode the target variable with LabelEncoder
le = LabelEncoder()
reviews_train['class'] = le.fit_transform(reviews_train['class'])

# Features
X = reviews_train.drop(['class'], axis=1) 

# Target variable
y = reviews_train['class']

# Split the dataset into a training and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print ("Training Dataset Shape:", X_train.shape, y_train.shape, "\n")
print ("Test Dataset Shape:", X_test.shape, y_test.shape, "\n")

Training Dataset Shape: (525, 10000) (525,) 

Test Dataset Shape: (225, 10000) (225,) 



## Testing the p-value feature selection method with SDG Classifier

In [5]:
# Select features
significant_features = select_features(X_train, y_train, 'p-value')
X_train_selected = X_train[significant_features]
X_test_selected = X_test[significant_features]
print("Number of features selected:", len(significant_features), "\n")

# Standarize the features
scaler = StandardScaler().fit(X_train_selected)
X_train_scaled = scaler.transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Create the linear model SGDclassifier
linear_clf = SGDClassifier()

# Train the model
linear_clf.fit(X_train_scaled, y_train)

# Predict the labels of the test set
y_pred = linear_clf.predict(X_test_scaled)

# Which average to choose???
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1 score:",metrics.f1_score(y_test, y_pred, average='macro', zero_division=1))
print("Precision:",metrics.precision_score(y_test, y_pred, average='macro', zero_division=1))
print("Recall:",metrics.recall_score(y_test, y_pred, average='macro', zero_division=1))

Number of features selected: 5719 

Accuracy: 0.6044444444444445
F1 score: 0.5778143555790615
Precision: 0.6466854256854256
Recall: 0.6301507936507936


## Testin all the feature selection methods with SDG Classifier

In [6]:
# Feature selection methods
feature_selection_methods = [None, 'p-value', 'variance', 'correlation']

for method in feature_selection_methods:
    
    if method is not None:
        print(f"\n--- Feature Selection Method: {method.upper()} ---")
    else:
        print(f"\n--- Feature Selection Method: None ---")

    significant_features = select_features(X_train, y_train, method)
    X_train_selected = X_train[significant_features]
    X_test_selected = X_test[significant_features]

    print("Number of features selected:", len(significant_features), "\n")

    # Standardize the features
    scaler = StandardScaler().fit(X_train_selected)
    X_train_selected = scaler.transform(X_train_selected)
    X_test_selected = scaler.transform(X_test_selected)

    # Create the linear model SGDclassifier
    linear_clf = SGDClassifier()

    # Train the model
    linear_clf.fit(X_train_selected, y_train)

    # Predict the labels of the test set
    y_pred = linear_clf.predict(X_test_selected)

    # Evaluate the performance
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("F1 score:", metrics.f1_score(y_test, y_pred, average='macro', zero_division=1))
    print("Precision:", metrics.precision_score(y_test, y_pred, average='macro', zero_division=1))
    print("Recall:", metrics.recall_score(y_test, y_pred, average='macro', zero_division=1))



--- Feature Selection Method: None ---
Number of features selected: 10000 

Accuracy: 0.6177777777777778
F1 score: 0.5759735046515232
Precision: 0.6403628038628039
Recall: 0.6188809523809524

--- Feature Selection Method: P-VALUE ---
Number of features selected: 5719 

Accuracy: 0.5688888888888889
F1 score: 0.5200481709552917
Precision: 0.5999517704517704
Recall: 0.578873015873016

--- Feature Selection Method: VARIANCE ---
Number of features selected: 3498 

Accuracy: 0.38222222222222224
F1 score: 0.3120535582408647
Precision: 0.37935930735930734
Recall: 0.3502777777777778

--- Feature Selection Method: CORRELATION ---
Number of features selected: 4746 

Accuracy: 0.3377777777777778
F1 score: 0.2822123937294216
Precision: 0.3509285714285714
Recall: 0.3092619047619048


## Performing the same tests with Random Forest Classifier

In [7]:
# Feature selection methods
feature_selection_methods = [None, 'p-value', 'variance', 'correlation']

for method in feature_selection_methods:
    
    if method is not None:
        print(f"\n--- Feature Selection Method: {method.upper()} ---")
    else:
        print(f"\n--- Feature Selection Method: None ---")

    significant_features = select_features(X_train, y_train, method)
    X_train_selected = X_train[significant_features]
    X_test_selected = X_test[significant_features]

    print("Number of features selected:", len(significant_features), "\n")

    # Standardize the features
    scaler = StandardScaler().fit(X_train_selected)
    X_train_selected = scaler.transform(X_train_selected)
    X_test_selected = scaler.transform(X_test_selected)

    # Create model Random Forest
    rf = RandomForestClassifier()

    # Train the model
    rf.fit(X_train_selected, y_train)

    # Predict the labels of the test set
    y_pred = rf.predict(X_test_selected)

    # Evaluate the performance
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("F1 score:", metrics.f1_score(y_test, y_pred, average='macro', zero_division=1))
    print("Precision:", metrics.precision_score(y_test, y_pred, average='macro', zero_division=1))
    print("Recall:", metrics.recall_score(y_test, y_pred, average='macro', zero_division=1))


--- Feature Selection Method: None ---
Number of features selected: 10000 

Accuracy: 0.49333333333333335
F1 score: 0.4581038844196739
Precision: 0.5412967032967032
Recall: 0.49233333333333335

--- Feature Selection Method: P-VALUE ---
Number of features selected: 5719 

Accuracy: 0.5288888888888889
F1 score: 0.5010717615717616
Precision: 0.6225555555555555
Recall: 0.5524444444444444

--- Feature Selection Method: VARIANCE ---
Number of features selected: 3498 

Accuracy: 0.41333333333333333
F1 score: 0.35049768205650556
Precision: 0.45782251082251085
Recall: 0.39699999999999996

--- Feature Selection Method: CORRELATION ---
Number of features selected: 4746 

Accuracy: 0.4444444444444444
F1 score: 0.3918923560100031
Precision: 0.5014975024975025
Recall: 0.44525396825396824
